In [ ]:
"""
A Script that plots the proportions of labeled class behaviors that fall within a certain magnitude.
The goal of this experiment is to determine if there is some more
"""

import torch
import time
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from data.swarmset import ContinuingDataset, SwarmDataset
from src.networks.embedding import NoveltyEmbedding
from src.networks.archive import DataAggregationArchive
from src.networks.ensemble import Ensemble
import numpy as np
from scipy import ndimage
import random
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

def CSVLineToVec(line):
    line_list = line.strip().replace("\n", "").split(",")
    float_list = []
    for i in line_list:
        float_list.append(float(i))
    float_list = np.array(float_list)
    return float_list

sampled_dataset = SwarmDataset("../data/full-mini", rank=0)
data = sampled_dataset

lim = 1000
controllers = []
classes = [-1 for i in range(lim)]
for i in range(lim):
    image, genome, behavior = sampled_dataset[i][0], sampled_dataset[i][1], sampled_dataset[i][2]
    for j in range(len(behavior)):
        if behavior[j] < 0.0:
            behavior[j] *= -1
    controllers.append(genome)

OUT = "../data/oracle"
with open(os.path.join(OUT, "original-hand-labeled-classes.txt"), "r") as f:
    lines = f.readlines()
    for line in lines:
        triplet = CSVLineToVec(line)
        classes[int(triplet[0])] = int(triplet[1])

random = []
not_random = []
rand_count, other_count = 0, 0
for i, label in enumerate(classes):
    v0_l, v0_r = controllers[i][0], controllers[i][1]
    v1_l, v1_r = controllers[i][2], controllers[i][3]
    v0_l, v0_r, v1_l, v1_r = round(v0_l, 1), round(v0_r, 1), round(v1_l, 1), round(v1_r, 1)

    k = 0.4
    max_elem_score = max(-min(controllers[i]), max(controllers[i]))
    max_elem_score = -max_elem_score if max_elem_score < k else max_elem_score

    k_2 = 0.65
    magnitude_score = np.linalg.norm(controllers[i])
    magnitude_score = -5 if magnitude_score < k_2 else magnitude_score

    k_3 = 0.3
    average_score = np.average(np.sqrt(np.power(controllers[i], 2)))
    average_score = -5 if average_score < k_3 else average_score

    # Sensor off magnitude (trial i)
    on_magnitude = (v0_l**2) + (v0_r**2)

    # Sensor on magnitude (trial i)
    off_magnitude = (v1_l**2) + (v1_r**2)

    # Spinning Detection (sensor off - trial ii)
    if v0_l == 0.0 and v0_r == 0.0:
        off_spin_variance = 1
        print("Gottem")
    else:
        denom = v0_l if v0_l != 0.0 else v0_r
        off_spin_variance = min(abs((v0_l + v0_r) / denom), 1.0)
    # mx = off_spin_variance

    # Spinning Detection (sensor on - trial ii)
    if v1_l == 0.0 and v1_r == 0.0:
        on_spin_variance = 0.0
        print("Gottem")
    else:
        denom = v1_l if v1_l != 0.0 else v1_r
        on_spin_variance = min(abs((v1_l + v1_r) / denom), 1)
    # mx = on_spin_variance

    # Mirror Property
    mirrored_controller = np.array([v0_l, v0_r, -v0_l, -v0_r])
    mirror_score = np.linalg.norm(mirrored_controller - controllers[i])
    k_m = 0.2
    mirror_score = -5 if mirror_score < k_m else mirror_score


    # Independence Property
    independent_controller = np.array([v0_l, v0_r, v0_l, v0_r])
    indep = np.linalg.norm(independent_controller - controllers[i])
    indep = -5 if indep < 0.3 else indep

    # mx = indep + mirror_score + 2*on_spin_variance + 2*off_spin_variance + on_magnitude + off_magnitude + max_elem_score + magnitude_score + average_score
    attributes = [
        indep,
        mirror_score ,
        on_spin_variance,
        off_spin_variance,
        on_magnitude,
        off_magnitude,
        max_elem_score,
        magnitude_score,
        average_score,
    ]



    # Modified
    weights = [0.8993,  0.7878,  1.7404,  0.7404,  0.4437,  0.3982, -0.1233,  0.0905, 1.1693]

    # Final-2 : includes hand-defined penalties for max_elem, magnitude_score, and average_score
    # weights = [3.8005,  3.1230,  2.1501,  1.3603, -2.7499, -2.7087, -1.8457,  0.1532, 8.1944]

    # 33.8% - 3.5%
    # weights = [5.3943,  4.5802,  3.3803,  1.7969, -4.1899, -3.9899, -7.3916,  2.5855, 10.1178]

    # 26% - 2%
    # weights = [4.0272,  3.5402,  2.4813,  1.3740, -1.0721, -1.1566, -6.2047, -0.2631, 4.4113]

    # 19% - 0.5%
    # weights = [0.8481,  0.7023,  1.2814,  0.5064,  0.1424,  0.1305, -0.2721,  0.0841, 1.3685]

    mx = np.dot(np.array(attributes), np.array(weights))

    spin_score = abs((v0_l + v0_r)) + abs((v1_l + v1_r))
    spin_score = -5 if spin_score < 0.5 else spin_score
    mx = spin_score + magnitude_score + max_elem_score + mirror_score + indep

    if mx < 4.0:
        if classes[i] == 0:
            rand_count += 1
        else:
            other_count += 1

    if mx > 15 and classes[i] == 0:
        print(controllers[i], classes[i])

    if label == 0:
        random.append(mx)
    else:
        not_random.append(mx)

print(f"RANDOM: {rand_count}, OTHER: {other_count}")

plt.hist(random, bins=30, alpha=0.5, label='Random Controllers')
plt.hist(not_random, bins=30, alpha=0.5, label='Other Controllers')
plt.legend(loc='upper left')
plt.axvline(x = 4.0, color = 'r', label = 'axvline - full height')
# plt.title("Network-informed Random Detection for Swarm Controllers")
plt.xlabel("Summed Scores from 5 Metrics")
plt.ylabel("Count")
plt.show()

# Also try hist2d with the second largest value from each controller

# EPSILON = 0.05
# total_counts = [0 for i in range(max(classes) + 1)]
# count = [0 for i in range(max(classes) + 1)]
# for i, vec in enumerate(controllers):
#     total_counts[classes[i]] += 1
#
#     nparr = np.array(vec)
#     # if np.linalg.norm(vec) > EPSILON:
#     #     count[classes[i]] += 1
#     for l in vec:
#         if EPSILON > l > -EPSILON:
#             print(vec)
#             count[classes[i]] += 1
#             break
#
# print(total_counts, count)